In [ ]:

#@title **Create User**
#@markdown Enter Username and Password

import os

username = "user" #@param {type:"string"}
password = "password" #@param {type:"string"}

#@markdown ![Final](https://i.postimg.cc/DyNhNjKF/image.png)

print("Create Users and manage them")

# Creating User
os.system(f"useradd -m {username}")

# Add user to user group sudo
os.system(f"adduser {username} sudo")
    
# Create a password for the user 'root'
os.system(f"echo '{username}:{password}' | sudo chpasswd")

# Change default shell from sh to bash
os.system("sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd")

print("User success is created and configured")


In [ ]:

#@title **RDP**
#@markdown It takes 4-5 minutes for installation

import os
import subprocess
#@markdown Visit http://remotedesktop.google.com/headless and Copy debian linux command after authentication

 CR = "" #@param {type:"string"}

  #@markdown Enter a pin of 6 digits or more
   Pin = 123456 #@param {type: "integer"}
#@markdown ![CRD](https://i.imgur.com/zYK1wKp.png)

class CRD:
    def __init__(self):
        os.system("apt update")
        self.installCRD()
        self.installDesktopEnvironment()
        self.installGoogleChorme()
        self.finish()

    @staticmethod
    def installCRD():
        print("Installing Chrome Remote Desktop")
        subprocess.run(['wget', 'https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['dpkg', '--install', 'chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def installDesktopEnvironment():
        print("Installing Desktop Environment")
        os.system("export DEBIAN_FRONTEND=noninteractive")
        os.system("apt install --assume-yes xfce4 desktop-base xfce4-terminal")
        os.system("bash -c 'echo \"exec /etc/X11/Xsession /usr/bin/xfce4-session\" > /etc/chrome-remote-desktop-session'")
        os.system("apt remove --assume-yes gnome-terminal")
        os.system("apt install --assume-yes xscreensaver")
        os.system("systemctl disable lightdm.service")

    @staticmethod
    def installGoogleChorme():
        print("Installing Google Chrome")
        subprocess.run(["wget", "https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(["dpkg", "--install", "google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def finish():
        print("Output")
        os.system(f"adduser {username} chrome-remote-desktop")
        command = f"{CRP} --pin={Pin}"
        os.system(f"su - {username} -c '{command}'")
        os.system("service chrome-remote-desktop start")
        print("Success, everything has been made and please open Chrome Remote Desktop")


try:
    if username:
        if CRP == "":
            print("Please enter the auth code from the link I provided")
        elif len(str(Pin)) < 6:
            print("Please enter a pin of at least 6 digit")
        else:
            CRD()
except NameError as e:
    print("Error : variable username empty")
    print("Create a user first!")

     


In [ ]:
#@title **Google Drive Mount**

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title **SSH (Optional)**

! pip install colab_ssh --upgrade &> /dev/null

Ngrok = False #@param {type:'boolean'}
Agro = False #@param {type:'boolean'}


#@markdown Copy authtoken from https://dashboard.ngrok.com/auth (must be from ngrok)
ngrokToken = "" #@param {type:'string'}


def runNGROK():
    from colab_ssh import launch_ssh
    from IPython.display import clear_output
    launch_ssh(ngrokToken, password)
    clear_output()

    print("ssh", username, end='@')
    ! curl -s http://localhost:4040/api/tunnels | python3 -c \
            "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'][6:].replace(':', ' -p '))"


def runAgro():
    from colab_ssh import launch_ssh_cloudflared
    launch_ssh_cloudflared(password=password)


try:
    if username:
        pass
    elif password:
        pass
except NameError:
    print("User not found, using username and password with 'root'")
    username='root'
    password='root'


if Agro and Ngrok:
    print("You cannot select all")
    print("Select only one of them")
elif Agro:
    runAgro()
elif Ngrok:
    if ngrokToken == "":
        print("Ngrok authtoken not found, please enter correctly")
    else:
        runNGROK()
else:
    print("Just choose one")


In [ ]:
#@title **Colab Shutdown**

#@markdown to turn off NGROK
NGROK = False #@param {type:'boolean'}

#@markdown To Unmount GDrive
GDrive = False #@param {type:'boolean'}

#@markdown To Create a Rest colab
Sleep = True #@param {type:'boolean'}

if NGROK:
    ! killall ngrok

if GDrive:
    with open('/content/unmount.py', 'w') as unmount:
        unmount.write("""from google.colab import drive
drive.flush_and_unmount()""")
    
    try:
        if user:
            ! runuser $user -c 'python3 /content/unmount.py'
    except NameError:
        print("Not installed with google drive")

if Sleep:
    from time import sleep
    sleep(43200)